In [20]:
import pgeocode
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import requests
from bs4 import BeautifulSoup
import re

In [58]:
def get_geodesic_info(zip1: int = None, zip2: int = None) -> int:
    """Retrieve LAT/LONG coordinates for two given zip codes
    
    Parameters:
    zip1 (int): Location A zipcode
    zip2 (int): Location B zipcode

    Returns:
    (int): dataframe containing geodesic information on zipcode pair
    """
    nomi = pgeocode.Nominatim('us') # Use United States Zip Codes
    # Extract Information
    return(pd.DataFrame(nomi.query_postal_code([str(zip1), str(zip2)])).drop(columns=['county_code', 'community_name', 'community_code', 'accuracy']))
def haversine_distance(geodesic_info: pd.DataFrame = None) -> int:
    """ Return distance (as crow files) in miles between two coordinates
    
    Parameters:
    geodesic_info (pd.DataFrame): Dataframe returned from get_geodesic_indo

    Returns:
    (float): Distance in miles between the two coordinates
    """
    geodesic_info = geodesic_info[['latitude', 'longitude']] # Extract Coordinates
    geodesic_info = geodesic_info.applymap(radians) # Convert to Radians
    # Calculate and Extract Haversine Distance
    distance = haversine_distances([geodesic_info.iloc[0,:].values, geodesic_info.iloc[1,:].values])[0][1]
    distance = distance * 3958.75587 # Multiply by Earth radius to get miles
    return(distance)

In [26]:
get_geodesic_info('80525', '83012')

,postal_code,country_code,place_name,state_name,state_code,county_name,latitude,longitude
0,80525,US,Fort Collins,Colorado,CO,Larimer,40.5384,-105.0547
1,83012,US,Moose,Wyoming,WY,Teton,43.7150,-110.7420


In [61]:
haversine_distance(get_geodesic_info(80525, 80524))

4.163244343914067

In [89]:
def get_postal_code(unitCode: str = None) -> int:
    """Retrieve the zipcode from a park unitcode

    Parameters:
    untiCode (str): 4-character park unitCode

    Returns:
    (int): Park zipcode
    """
    if unitCode is None: # Ensure unitCode provided
        return
    url = str("https://www.nps.gov/" + unitCode + "/contacts.htm") # Build URL
    page = requests.get(url) # Access URL
    soup = BeautifulSoup(page.content, 'html.parser') # Retrieve html information
    mailing_information = soup.find('div', class_='mailing-address').text.replace('\n', ' ') # Find Mailing Information
    mailing_information = ' '.join(mailing_information.split()) # Remove Whitespace
    # Matching ex: 'FL 33034-6733'
    text = re.search(r'\s?[A-Z]{2}\s(\d{5})(-\d{4})?', mailing_information) # Search for the zipcode information 
    if text is None: # Error no Match
        print("ERROR: No REGEX MATCH")
        return
    return(int(text.group(1))) # Return 5 digit zipcode only

In [90]:
get_postal_code('romo')

80517